In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from advi.model import ADVIModel
from models.ard import Ard
from models.ard import sep_params
#from models.ard import joint_log_prob, return_initial_state, sep_params, log_likelihood, some_kind_of_loss
# NEXT TO DO MUST CHANGE INITAL CHAIN STATES TO SAMPLE FROM DISTRIBUTION
# Target distribution is proportional to: `exp(-x (1 + x))`.

# Initialize the HMC transition kernel.



/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def make_training_data(num_samples, dims, sigma):
  x = np.random.randn(dims, num_samples).astype(np.float64)
  w = sigma * np.random.randn(1, dims).astype(np.float64)
  noise = np.random.randn(num_samples).astype(np.float64)
  w[:,:int(dims/2)] = 0
  y = w.dot(x) + noise
    
  return y, x, w

def sep_training_test(y,x,test):
  y_train = y[:,test:]
  x_train = x[:,test:]
  
  y_test = y[:,:test]
  x_test = x[:,:test]
  return y_train, y_test, x_train, x_test
num_features = 250

y, x, w = make_training_data(200, num_features, 2)
print("y shape", y.shape)
print("x shape", x.shape)
print("w shape", w.shape)


('y shape', (1, 200))
('x shape', (250, 200))
('w shape', (1, 250))


In [3]:
y_train, y_test, x_train, x_test = sep_training_test(y,x,50)
print(y_train.shape)
print(y_test.shape)
print(x_train.shape)
print(x_test.shape)



(1, 150)
(1, 50)
(250, 150)
(250, 50)


In [4]:
model = Ard(num_features)
initial_chain_state = model.return_initial_state()
wstart, tau, alpha = sep_params(initial_chain_state, num_features)


In [5]:
def state_to_log_pred(states, y_test, x_test):
    sample_mean = tf.reduce_mean(states, axis=[0])
    return model.log_likelihood(y_test, x_test, states)

def state_to_loss(states, y_test, x_test):
    sample_mean = tf.reduce_mean(states, axis=[0])
    w, _, _ = sep_params(states, num_features)
    return model.some_kind_of_loss(y_test, x_test, w)

state_to_loss(initial_chain_state, y_test, x_test)

<tf.Tensor: shape=(), dtype=float64, numpy=4062.9023260231247>

In [6]:
summary_writer = tf.compat.v2.summary.create_file_writer('/tmp/summary_chain', flush_millis=10000)
def trace_fn(state, results):
  with tf.compat.v2.summary.record_if(tf.equal(results.step % 10, 0)):
    tf.compat.v2.summary.scalar("loss", state_to_loss(state, y_test, x_test), step=tf.cast(results.step, tf.int64))
    tf.compat.v2.summary.scalar("log pred", state_to_log_pred(state, y_test, x_test), step=tf.cast(results.step, tf.int64))
    return ()

In [7]:
num_results = int(10000)
num_burnin_steps = int(800)
adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(
    tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=model.joint_log_prob,
        num_leapfrog_steps=3,
        step_size=0.1),
    num_adaptation_steps=int(num_burnin_steps * 0.8))

nuts = tfp.mcmc.NoUTurnSampler(
    target_log_prob_fn=model.joint_log_prob, step_size=1., max_tree_depth=10, max_energy_diff=1000.0,
    unrolled_leapfrog_steps=1, parallel_iterations=10, seed=None, name=None)

@tf.function
def run_chain():
  print("running chain")
  # Run the chain (with burn-in).
  states, is_accepted = tfp.mcmc.sample_chain(
      num_results=num_results,
      num_burnin_steps=num_burnin_steps,
      current_state=initial_chain_state,
      kernel=adaptive_hmc,
    trace_fn=trace_fn)
  return states, is_accepted

    

@tf.function
def run_chain_nuts():
  print("running chain")
  # Run the chain (with burn-in).
  states, is_accepted = tfp.mcmc.sample_chain(
      num_results=20,
      num_burnin_steps=num_burnin_steps,
      current_state=initial_chain_state,
      kernel=nuts)
  return states, is_accepted

In [ ]:
def run_advi(nsteps, step_size, dim, log_prob, bijector,m):
    advi = ADVIModel(dim, log_prob, bijector,m)
    for t in range(nsteps):
        grad_mu, grad_omega = advi.gradients()
        advi.mu = tf.add(advi.mu, step_size * grad_mu)
        advi.omega = tf.add(advi.omega, step_size * grad_omega)
        if(t%1 == 0):
            theta_intermediate = advi.sample()
            theta_intermediate = tf.squeeze(theta_intermediate)
            w, _, _ = sep_params(theta_intermediate, num_features)
            loss = model.some_kind_of_loss(y_test, x_test, w)
            likelihood = model.log_likelihood(y_test, x_test, theta_intermediate)
            #print("loss", loss)
            print("likelihood", likelihood)
        
    # sample from eta distribution currently just once,
    # using current values of mu and sigma to return trained params theta
    theta = advi.sample()
    return theta

In [ ]:
joint_log_prob2 = lambda *args: model.joint_log_prob(y_train, x_train, *args)
target = model.joint_log_prob
bij = tfp.bijectors.Log()
theta = run_advi(1000,0.00001,
                 model.num_params,
                 joint_log_prob2, bij,
                 10)

('likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-325.75081402814277>)
('likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-308.38231124036423>)
('likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-300.41301657159636>)
('likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-294.5950172860092>)
('likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-290.78106512500057>)
('likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-285.1858858806987>)
('likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-282.2613830183094>)
('likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-286.8802385058618>)
('likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-279.14968789925086>)
('likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-276.21205757406585>)
('likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-280.6932262556765>)
('likelihood', <tf.Tensor: shape=(), dtype=float64, numpy=-273.9422769508329>)
('likelihood', <tf.Tensor: shape=(), dtype=flo

In [ ]:

start_loss = state_to_loss(initial_chain_state, y_test, x_test)
print("start loss", start_loss)
print("theta", theta.shape)
theta = tf.squeeze(theta)
w, _, _ = sep_params(theta, num_features)
loss = model.some_kind_of_loss(y_test, x_test, w)

print("loss", loss)



In [ ]:
initial_chain_state = return_initial_state(num_features)
sample_mean = tf.reduce_mean(states, axis=[0])
wend, alpha, tau = sep_params(sample_mean, num_features)

print("start", wstart)
loss_start = some_kind_of_loss(wstarty_test, x_test, )
print("loss start", loss_start)

print("actual",w)
print("end", wend)
print("start", wstart)
loss_end = some_kind_of_loss(wend,y_test, x_test, )
print("loss start", loss_start)
print("loss end", loss_end)


In [ ]:
sample_mean = tf.reduce_mean(states, axis=[0])
wend, alpha, tau = sep_params(sample_mean, num_features)

